# M5 COMPETITION: PART 3: ML

## 1. IMPORT

In [39]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from plotly import tools, subplots
import plotly.graph_objs as go
import plotly.express as px

from pandas.plotting import scatter_matrix

from IPython.display import display

from catboost import CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split

## 2. READ DATA

In [40]:
calendar_df = pd.read_csv('calendar.csv')

sell_prices_df = pd.read_csv('sell_prices.csv')

data_df = pd.read_csv('data.csv')

## 3. DATA WRANGLING

In [41]:
print('Size of calendar_df data is: ', calendar_df.shape)
print('Size of data_df data is: ', data_df.shape)
print('Size of sell_prices_df data is: ', sell_prices_df.shape)

Size of calendar_df data is:  (1969, 14)
Size of data_df data is:  (1969, 19)
Size of sell_prices_df data is:  (6841121, 4)


### 3.1 Calendar_df

In [42]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [43]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non-null      object
 11  snap_CA       1969 non-null   int64 
 12  snap_TX       1969 non-null   int64 
 13  snap_WI       1969 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


In [44]:
#Missing Values

total_values = calendar_df.isnull().sum().sort_values(ascending = False)
percentage = (calendar_df.isnull().sum()/calendar_df.isnull().count()*100).sort_values(ascending = False)

missing_calendar = pd.concat([total_values, percentage], axis = 1, keys = ['Total Values', 'Percentage'])
missing_calendar

,Total Values,Percentage
event_name_2,1964,99.746064
event_type_2,1964,99.746064
event_name_1,1807,91.772473
event_type_1,1807,91.772473
date,0,0.000000
wm_yr_wk,0,0.000000
weekday,0,0.000000
wday,0,0.000000
month,0,0.000000
year,0,0.000000


### 3.2 sell_prices_df

In [45]:
sell_prices_df.head(1000)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
995,CA_1,HOBBIES_1_005,11437,2.88
996,CA_1,HOBBIES_1_005,11438,2.88
997,CA_1,HOBBIES_1_005,11439,2.88
998,CA_1,HOBBIES_1_005,11440,2.88


In [46]:
sell_prices_df.item_id.value_counts()

FOODS_3_587        2820
HOUSEHOLD_1_177    2820
HOUSEHOLD_2_283    2820
HOBBIES_1_337      2820
FOODS_1_032        2820
                   ... 
HOUSEHOLD_1_308     642
HOUSEHOLD_1_159     623
HOUSEHOLD_1_242     600
FOODS_3_296         592
FOODS_2_379         530
Name: item_id, Length: 3049, dtype: int64

In [47]:
sell_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [48]:
total_values = sell_prices_df.isnull().sum().sort_values(ascending = False)
percentage = (sell_prices_df.isnull().sum()/sell_prices_df.isnull().count()*100).sort_values(ascending = False)

missing_sell_prices = pd.concat([total_values, percentage], axis = 1, keys = ['Total Values', 'Percentage'])
missing_sell_prices

,Total Values,Percentage
store_id,0,0.0
item_id,0,0.0
wm_yr_wk,0,0.0
sell_price,0,0.0


### 3.3 data_df

In [49]:
data_df.head(100)

,date,Hobbies_CA_1,Household_1_CA_1,Household_2_CA_1,Foods_1_CA_1,Foods_2_CA_1,Foods_3_CA_1,Hobbies_CA_2,Household_1_CA_2,Household_2_CA_2,Foods_1_CA_2,Foods_2_CA_2,Foods_3_CA_2,Hobbies_CA_3,Household_1_CA_3,Household_2_CA_3,Foods_1_CA_3,Foods_2_CA_3,Foods_3_CA_3
0,2011-01-29,389,105,114,208,305,1799,374,169,143,63,274,1260,383,160,176,188,307,2029
1,2011-01-30,381,104,110,210,347,1808,238,152,154,47,209,1066,310,200,204,240,341,2035
2,2011-01-31,314,96,77,117,153,1118,238,105,97,39,123,759,271,168,148,183,278,1535
3,2011-02-01,303,90,66,122,183,1265,233,84,92,49,130,904,309,174,160,185,337,1900
4,2011-02-02,184,59,77,109,209,1217,198,73,104,20,112,787,233,141,150,162,290,1699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2011-05-04,236,84,52,111,158,1116,152,101,85,42,98,697,288,191,132,183,253,1478
96,2011-05-05,348,85,62,102,134,894,162,90,92,53,110,687,243,166,128,178,233,1498
97,2011-05-06,285,80,78,178,189,1204,244,112,120,42,128,773,319,190,144,193,258,1501
98,2011-05-07,395,170,105,163,234,1456,228,187,138,46,140,805,289,239,144,244,252,1678


We need to transform the dataset in order to make it usable ! we must blend all the 19 Object per Store into one row, and the following for each date. The function pd.melt provide such a solution. 

In [50]:
data_melt_df = pd.melt(data_df, id_vars = ['date'], value_vars = ['Hobbies_CA_1', 'Household_1_CA_1', 'Household_2_CA_1', 'Foods_1_CA_1', 'Foods_2_CA_1', 'Foods_3_CA_1', 'Hobbies_CA_2', 'Household_1_CA_2', 'Household_2_CA_2', 'Foods_1_CA_2', 'Foods_2_CA_2', 'Foods_3_CA_2', 'Hobbies_CA_3', 'Household_1_CA_3', 'Household_2_CA_3', 'Foods_1_CA_3', 'Foods_2_CA_3', 'Foods_3_CA_3'], ignore_index = False)
data_melt_df

,date,variable,value
0,2011-01-29,Hobbies_CA_1,389
1,2011-01-30,Hobbies_CA_1,381
2,2011-01-31,Hobbies_CA_1,314
3,2011-02-01,Hobbies_CA_1,303
4,2011-02-02,Hobbies_CA_1,184
...,...,...,...
1964,2016-06-15,Foods_3_CA_3,1796
1965,2016-06-16,Foods_3_CA_3,1850
1966,2016-06-17,Foods_3_CA_3,2029
1967,2016-06-18,Foods_3_CA_3,2443


In [51]:
data_melt_df = data_melt_df.sort_index()

In [52]:
data_melt_df[data_melt_df['date'] == '2011-01-29']

,date,variable,value
0,2011-01-29,Hobbies_CA_1,389
0,2011-01-29,Foods_1_CA_1,208
0,2011-01-29,Foods_2_CA_1,305
0,2011-01-29,Foods_3_CA_3,2029
0,2011-01-29,Foods_3_CA_1,1799
0,2011-01-29,Hobbies_CA_2,374
0,2011-01-29,Household_1_CA_2,169
0,2011-01-29,Household_2_CA_1,114
0,2011-01-29,Household_2_CA_2,143
0,2011-01-29,Foods_2_CA_3,307


In [53]:
data_melt_df.head(30)

,date,variable,value
0,2011-01-29,Hobbies_CA_1,389
0,2011-01-29,Foods_1_CA_1,208
0,2011-01-29,Foods_2_CA_1,305
0,2011-01-29,Foods_3_CA_3,2029
0,2011-01-29,Foods_3_CA_1,1799
0,2011-01-29,Hobbies_CA_2,374
0,2011-01-29,Household_1_CA_2,169
0,2011-01-29,Household_2_CA_1,114
0,2011-01-29,Household_2_CA_2,143
0,2011-01-29,Foods_2_CA_3,307


Now we must create the feature 'store' (e.g: CA_1, CA_2, CA_3), ' and 'product (e.g: HOBBIES, HOUSEHOLD_1...)

In [54]:
#Match the store
data_melt_df['store'] = data_melt_df.variable.str.findall('CA_+[0-9]').str[0]

#Match the product
data_melt_df['product'] = data_melt_df.variable.str.findall('(.*?)\_CA').str[0]

In [55]:
data_melt_df = data_melt_df.drop(columns = ['variable'])
data_melt_df.head()

,date,value,store,product
0,2011-01-29,389,CA_1,Hobbies
0,2011-01-29,208,CA_1,Foods_1
0,2011-01-29,305,CA_1,Foods_2
0,2011-01-29,2029,CA_3,Foods_3
0,2011-01-29,1799,CA_1,Foods_3


The data_df is properly transformed. We can pursue.

## 4. MERGING

Time to merge the data_melt_df with the calendar_df in order to get some features.

In [56]:
#Do a left join on ID 'date'.

data_merge_df = data_melt_df.merge(calendar_df, on = ['date'], how = 'left')
data_merge_df.head()

,date,value,store,product,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,389,CA_1,Hobbies,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-29,208,CA_1,Foods_1,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-29,305,CA_1,Foods_2,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
3,2011-01-29,2029,CA_3,Foods_3,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
4,2011-01-29,1799,CA_1,Foods_3,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0


In [57]:
#Here we replace the the nan values in events with 'no_events'. 
data_merge_df.fillna('no_events', inplace = True)

In [58]:
data_final = data_merge_df.drop(columns = ['date', 'd', 'wday'])
data_final.head()

,value,store,product,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,389,CA_1,Hobbies,11101,Saturday,1,2011,no_events,no_events,no_events,no_events,0,0,0
1,208,CA_1,Foods_1,11101,Saturday,1,2011,no_events,no_events,no_events,no_events,0,0,0
2,305,CA_1,Foods_2,11101,Saturday,1,2011,no_events,no_events,no_events,no_events,0,0,0
3,2029,CA_3,Foods_3,11101,Saturday,1,2011,no_events,no_events,no_events,no_events,0,0,0
4,1799,CA_1,Foods_3,11101,Saturday,1,2011,no_events,no_events,no_events,no_events,0,0,0


In [59]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35442 entries, 0 to 35441
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   value         35442 non-null  int64 
 1   store         35442 non-null  object
 2   product       35442 non-null  object
 3   wm_yr_wk      35442 non-null  int64 
 4   weekday       35442 non-null  object
 5   month         35442 non-null  int64 
 6   year          35442 non-null  int64 
 7   event_name_1  35442 non-null  object
 8   event_type_1  35442 non-null  object
 9   event_name_2  35442 non-null  object
 10  event_type_2  35442 non-null  object
 11  snap_CA       35442 non-null  int64 
 12  snap_TX       35442 non-null  int64 
 13  snap_WI       35442 non-null  int64 
dtypes: int64(7), object(7)
memory usage: 4.1+ MB


In [60]:
#Transform the columns in their right categories.
cols = ['wm_yr_wk', 'month', 'year', 'snap_CA', 'snap_TX', 'snap_WI']
data_final[cols] = data_final[cols].astype(object)

In [61]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35442 entries, 0 to 35441
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   value         35442 non-null  int64 
 1   store         35442 non-null  object
 2   product       35442 non-null  object
 3   wm_yr_wk      35442 non-null  object
 4   weekday       35442 non-null  object
 5   month         35442 non-null  object
 6   year          35442 non-null  object
 7   event_name_1  35442 non-null  object
 8   event_type_1  35442 non-null  object
 9   event_name_2  35442 non-null  object
 10  event_type_2  35442 non-null  object
 11  snap_CA       35442 non-null  object
 12  snap_TX       35442 non-null  object
 13  snap_WI       35442 non-null  object
dtypes: int64(1), object(13)
memory usage: 4.1+ MB


## 5. CATBOOST

In [62]:
y = data_final['value']
X = data_final.drop('value', axis = 1)

cat_features = np.where(X.dtypes == np.object)[0]

data = train_test_split(X, y, test_size = 0.20, random_state = 0)
X_train, X_validation, y_train, y_validation = data

train_pool = Pool(data = X_train, label = y_train, cat_features = cat_features)
validation_pool = Pool(data = X_validation, label = y_validation, cat_features = cat_features)

C:\Users\decom\AppData\Local\Temp/ipykernel_10868/2901673634.py:4: DeprecationWarning:

`np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [66]:
cat = CatBoostRegressor(iterations=1000, depth=14, learning_rate = 0.1, verbose = 10, loss_function= 'RMSE', l2_leaf_reg=3, border_count = 254, one_hot_max_size = 400)

cat.fit(train_pool, eval_set = validation_pool, plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 550.9100196	test: 557.8860295	best: 557.8860295 (0)	total: 48ms	remaining: 48s
10:	learn: 237.2467121	test: 238.2193407	best: 238.2193407 (10)	total: 1.53s	remaining: 2m 17s
20:	learn: 137.5912603	test: 140.4484551	best: 140.4484551 (20)	total: 3.53s	remaining: 2m 44s
30:	learn: 106.5265271	test: 112.6838248	best: 112.6838248 (30)	total: 5.72s	remaining: 2m 58s
40:	learn: 93.1400405	test: 101.5535310	best: 101.5535310 (40)	total: 8.17s	remaining: 3m 11s
50:	learn: 85.7084431	test: 95.6462166	best: 95.6462166 (50)	total: 10.6s	remaining: 3m 16s
60:	learn: 80.8103771	test: 91.9384958	best: 91.9384958 (60)	total: 13s	remaining: 3m 20s
70:	learn: 76.8823385	test: 89.7729039	best: 89.7729039 (70)	total: 15.7s	remaining: 3m 25s
80:	learn: 73.3988712	test: 87.2808769	best: 87.2808769 (80)	total: 18.2s	remaining: 3m 26s
90:	learn: 70.8879573	test: 85.7149657	best: 85.7149657 (90)	total: 20.6s	remaining: 3m 25s
100:	learn: 69.0165461	test: 84.4988863	best: 84.4988863 (100)	total: 23.1